In [1]:
import re
import string

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

import gensim.downloader as api

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [2]:
# Загружаем из nltk стоп слова для русского и английского языка
stop1 = list(stopwords.words('english'))
stop2 = list(stopwords.words('russian'))
stop = stop1 + stop2

lemmatizer = WordNetLemmatizer()

# Функция для предобработки. Удаление знаков пуктуации, приведение к нижнему регистру, лемматизация.
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[.,"\'-?:!;]', "", line)
    line = nltk.word_tokenize(line)
    line = ' '.join([lemmatizer.lemmatize(w) for w in line])
    return line

In [3]:
# Читаем наш датасет.
df = pd.read_csv('train.csv')

In [4]:
# Посмотрим на данные.
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497819 entries, 0 to 497818
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   pair_id       497819 non-null  int64 
 1   name_1        497819 non-null  object
 2   name_2        497819 non-null  object
 3   is_duplicate  497819 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 15.2+ MB


In [6]:
# Посмотрим сколько одинаковых пар, а сколько разных.
df.is_duplicate.value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

In [7]:
# Разбиваем датафрейм на на части с копиями и без них.
df_0 = df[df['is_duplicate'] == 0]
df_1 = df[df['is_duplicate'] == 1]

name_1_df_0 = list(df_0['name_1'])
name_2_df_0 = list(df_0['name_2'])

name_1_df_1 = list(df_1['name_1'])
name_2_df_1 = list(df_1['name_2'])

In [8]:
# Объединим два столбца, посмотрим сколько всего названий.
text = list(df['name_1']) + list(list(df['name_2']))

print(len(text))

995638


In [9]:
# Оставим только уникальные названия.
text_unic = list(set(text))

print("Количество уникальных названий  " , len(text_unic))

# Разобъем все названия на отдельные слова
words = []
for string in text_unic:
    subwords = preprocessing(string).split()
    for word in subwords:
        words.append(word)

print("Количество слов" , len(words))
words_unic = list(set(words))
print("Количество уникальных слов ", len(words_unic))

Количество уникальных названий   18022
Количество слов 73720
Количество уникальных слов  16046


In [10]:
print(preprocessing(text[12]))

bestocean worldwide logistics inc


In [20]:
# Оставим только слова длиннее 1 символа.
filtered_words = [w for w in words_unic if len(w) > 1]

# Загрузим предобученную модель word2vec на полтора гигабайта
glove = api.load("glove-twitter-200")

In [21]:
# Функция делает эмбеддинг из строки, как сумму эмбеддингов входящих в строку слов
def sentence_embedding(string):
    sentence_list = [glove.get_vector(w) if w in glove else np.zeros(200) for w in string.split()  ]
    result = np.sum(np.array(sentence_list), axis=0)
    return result

In [24]:
# Пример текста. Предобработаем его, вычислим эмбеддинги, а потом посмотрим на косинусное сходство.
text1 = preprocessing('Carlisle Coatings & Waterproofing, Inc.' )
text2 = preprocessing('Carlisle Coatings & Wtrprfng')

print(text1)
print(text2)
print()

text1_vec = sentence_embedding(text1)
text2_vec = sentence_embedding(text2)


text2_1 = preprocessing('Iko Industries Ltd.' )
text2_2 = preprocessing('Enormous Industrial Trade Pvt., Ltd.')

print(text2_1)
print(text2_2)

text2_1_vec = sentence_embedding(text2_1)
text2_2_vec = sentence_embedding(text2_2)

carlisle coating & waterproofing inc
carlisle coating & wtrprfng

iko industry ltd
enormous industrial trade pvt ltd


In [25]:
# Посмотрим на косинусное сходство двух текстов.
print(cosine_similarity(text1_vec.reshape(1, -1), text2_vec.reshape(1, -1)))
print()
print(cosine_similarity(text2_1_vec.reshape(1, -1), text2_2_vec.reshape(1, -1)))

[[0.84703398]]

[[0.67530155]]


In [15]:
# Вычислим косинусное сходство для всех пар из нашего датасета.
pair_0 = [cosine_similarity(sentence_embedding(item[0]).reshape(1, -1), sentence_embedding(item[1]).reshape(1, -1)) for item in zip(name_1_df_0 ,name_2_df_0 )]
pair_1 = [cosine_similarity(sentence_embedding(item[0]).reshape(1, -1), sentence_embedding(item[1]).reshape(1, -1)) for item in zip(name_1_df_1 ,name_2_df_1 )]  

In [16]:
# Посмотрим на косинусное сходство первых 50 непохожиш пар.
print(pair_0[:50])

[array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.89154127]]), array([[0.]]), array([[0.]]), array([[0.]])]


In [17]:
# Посмотрим на косинусное сходство первых 50 похожиш пар.
print(pair_1[:50])

[array([[1.]]), array([[0.]]), array([[0.]]), array([[0.33409574]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[1.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.77249323]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]])]


In [18]:
print(len(pair_0))
print(len(pair_1))

494161
3658


In [19]:
# Выведем долю пар схожих и отличных по косинусному сходству по порогу 0.3
pair_0_trashold_05 = [i for i in pair_0 if i < 0.3]
pair_1_trashold_05 = [i for i in pair_1 if i>= 0.3]

print(len(pair_0_trashold_05), "  ",len(pair_0_trashold_05)/len(pair_0)*100)

print(len(pair_1_trashold_05), "  ",len(pair_1_trashold_05)/len(pair_1)*100)

486853    98.52112975325855
75    2.050300710770913
